<a href="https://colab.research.google.com/github/tanupriya9102/metro_cc/blob/main/ChatBotBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from torch.utils.data import TensorDataset
from transformers import DistilBertTokenizer
# from transformers import BertTokenizer

In [ ]:
from google.colab import files
upload= files.upload()

Saving chatbot.xlsx to chatbot.xlsx


In [ ]:
import pandas as pd
import torch
# from transformers import BertTokenizer, BertForQuestionAnswering

In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = TFDistilBertModel.from_pretrained("distilbert-base-uncased", output_hidden_states=True)



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
# import pandas as pd
# import numpy as np
from scipy.spatial.distance import cosine
# from transformers import DistilBertTokenizer, TFDistilBertModel

# # Load the DistilBERT tokenizer and model
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# model = TFDistilBertModel.from_pretrained("distilbert-base-uncased", output_hidden_states=True)

# Load the FAQ questions and answers from the Excel file
df = pd.read_excel("chatbot.xlsx")
faq_dict = {}
for index, row in df.iterrows():
    question = row["Question"]
    answer = row["Answer"]
    faq_dict[question] = answer

def get_contextual_embeddings(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True)
    inputs = tf.constant([input_ids])
    outputs = model(inputs)
    hidden_states = outputs.last_hidden_state
    contextual_embeddings = tf.reduce_mean(hidden_states, axis=1).numpy()
    return contextual_embeddings

def find_most_similar_question(input_question, faq_dict):
    input_embeddings = get_contextual_embeddings(input_question)

    max_similarity = -1
    most_similar_question = None

    for question in faq_dict:
        question_embeddings = get_contextual_embeddings(question)

        similarity = 1 - cosine(input_embeddings.flatten(), question_embeddings.flatten())

        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_question = question

    return most_similar_question, max_similarity


In [ ]:
def main():
    similarity_threshold = 0.85

    while True:
        # Ask the user a question
        question = input("What is your question? ")

        if question.lower() == "exit":
            print("Goodbye!")
            break

        # Find the most similar question in the dataset
        most_similar_question, similarity_score = find_most_similar_question(question, faq_dict)

        if most_similar_question is not None:
            if similarity_score < similarity_threshold:
                print("Sorry, I couldn't find a similar question in the dataset.")
            else:
                # Get the answer for the most similar question
                answer = faq_dict[most_similar_question]
                print(answer)
        else:
            print("Sorry, I couldn't find a similar question in the dataset.")

if __name__ == "__main__":
    main()


What is your question? What is a QR ticket?
QR code-paper tickets have been introduced on all lines for traveling in DMRC.
What is your question? What do you mean by qr ticket?
QR code-paper tickets have been introduced on all lines for traveling in DMRC.
What is your question? exit
Goodbye!
